In [16]:
import copy

def manhattan_distance(state, goal):
    # map each tile in goal to its (i,j) position
    goal_pos = {}
    for i in range(3):
        for j in range(3):
            goal_pos[goal[i][j]] = (i, j)

    dist = 0
    for i in range(3):
        for j in range(3):
            tile = state[i][j]
            if tile == '*':
                continue
            gi, gj = goal_pos[tile]  # store the indexes of the tile in the goal state
            dist += abs(i - gi) + abs(j - gj)
    return dist

def misplaced_tiles(state, goal):
    misplacedCount = 0
    for i in range(3):
        for j in range(3):
            if state[i][j] != '*' and state[i][j] != goal[i][j]:
                misplacedCount += 1
    return misplacedCount

def nilsson_sequence_score(state, goal, detailed=False):
    md = manhattan_distance(state, goal)

    pos_map = {}
    numbering = [(0,0),(0,1),(0,2),(1,2),(2,2),(2,1),(2,0),(1,0),(1,1)]
    for idx, (i,j) in enumerate(numbering):
        if state[i][j] != '*':
            pos_map[state[i][j]] = idx

    goal_order = ['1','2','3','4','5','6','7','8']

    seq_score = 0
    for k in range(8):
        tile = goal_order[k]
        next_tile = goal_order[(k+1) % 8]

        if tile in pos_map and next_tile in pos_map:
            if pos_map[next_tile] != (pos_map[tile] + 1) % 8:
                seq_score += 2

    cx, cy = numbering[8]
    if state[cx][cy] != '*':
        seq_score += 1

    total = md + 3 * seq_score

    if detailed:
        return total, md, seq_score
    return total


def compute_heuristics(state, goal):
    manhattan = manhattan_distance(state, goal)
    misplaced = misplaced_tiles(state, goal)
    return manhattan, misplaced


class Node:
    def __init__(self, state, g=0, h=0, parent=None):
        self.state = state      # puzzle configuration
        self.g = g              # cost so far
        self.h = h              # heuristic cost
        self.f = g + h          # f = g + h (A* score) [total cost]
        self.parent = parent    # reference to previous node

    def find(self):
        for i in range(3):
            for j in range(3):
                if self.state[i][j] == '*':
                    return i, j

    def move(self, x0, y0, x1, y1):
        if 0 <= x1 <= 2 and 0 <= y1 <= 2:
            tempstate = copy.deepcopy(self.state)
            tempstate[x0][y0], tempstate[x1][y1] = tempstate[x1][y1], tempstate[x0][y0]
            return tempstate
        return None

    # NOTE: generate returns children with g incremented; h will be computed later
    def generate(self, movement):
        x, y = self.find()
        if movement == 'r':
            new_state = self.move(x, y, x, y + 1)
        elif movement == 'l':
            new_state = self.move(x, y, x, y - 1)
        elif movement == 'u':
            new_state = self.move(x, y, x - 1, y)
        elif movement == 'd':
            new_state = self.move(x, y, x + 1, y)
        else:
            return None

        if new_state is not None:
            # create child node, increment g by 1, set parent to current node
            return Node(new_state, g=self.g + 1, parent=self)
        return None

    def __eq__(self, other):
        return isinstance(other, Node) and self.state == other.state

    def __repr__(self):
        return f"Node(f={self.f}, g={self.g}, h={self.h})"


class Puzzle:
    def __init__(self, heuristic, heuristic_name):
        self.open = []         
        self.closed = []        
        self.goal_state = None
        self.heuristic = heuristic  
        self.heuristic_name = heuristic_name
        self.nodes_generated = 0

    def read_input(self, filename="astar_in.txt"):
        with open(filename, 'r') as file:
            lines = file.readlines()
            lines = [line.strip().split(" ") for line in lines]

        start_state = lines[1:4]
        goal_state = lines[5:8]
        self.goal_state = goal_state
        return start_state, goal_state

    def init_start_node(self, start_state, goal_state):
        g_s = 0
        h_s = self.heuristic(start_state, goal_state)
        start = Node(start_state, g=g_s, h=h_s)
        self.open.append(start)

    def is_open_empty(self):
        return not self.open

    def get_best_node(self):
        min_f = min(node.f for node in self.open)

        candidates = [node for node in self.open if node.f == min_f]
        goal_candidates = [node for node in candidates if self.is_goal(node)]


        # picking the goal node if available, else any candidate
        if goal_candidates:
            n = goal_candidates[0]
        else:
            n = candidates[0]

        # move n from OPEN to CLOSED
        self.open.remove(n)
        self.closed.append(n)
        return n

    def display_solution(self, path):
        print("\nSolution Found!")
        print(f"Number of moves: {len(path) - 1}\n")
        
        for step, node in enumerate(path):
            if self.heuristic_name == "Nilsson's Sequence Score":
                h_val, p_val, s_val = nilsson_sequence_score(node.state, self.goal_state, detailed=True)
                print(f"Step {step}: f(n)={node.f}, g(n)={node.g}, h(n)={h_val}, P(n)={p_val}, S(n)={s_val}")
            else:
                h_val = self.heuristic(node.state, self.goal_state)
                print(f"Step {step}: f(n)={node.f}, g(n)={node.g}, h(n)={h_val}")
            for row in node.state:
                print(" ".join(row))
            print()

    def is_goal(self, node):
        return node.state == self.goal_state

    def reconstruct_path(self, node):
        path = []
        current = node
        while current:
            path.append(current)
            current = current.parent
        path.reverse()
        return path

    def expand_node(self, node):
        children = []
        directions = ['r', 'l', 'u', 'd']

        for move in directions:
            child = node.generate(move)
            if child is not None:
                # compute heuristic for child
                child.h = self.heuristic(child.state, self.goal_state)
                child.f = child.g + child.h
                children.append(child)
                self.nodes_generated += 1
        return children

    def add_children_to_open(self, children, parent):
        for child in children:
            # Searching existing node in OPEN and CLOSED
            open_match = next((c for c in self.open if c.state == child.state), None)
            closed_match = next((c for c in self.closed if c.state == child.state), None)

            if open_match is None and closed_match is None:
                # add the new nodes to OPEN
                self.open.append(child)
                child.parent = parent
            else:
                # it already exists in OPEN or CLOSED
                existing = open_match if open_match else closed_match

                if child.f < existing.f:
                    # update existing node with better value
                    existing.g = child.g
                    existing.h = child.h
                    existing.f = child.f
                    existing.parent = parent

                    if closed_match:
                        # move it back to OPEN
                        self.closed.remove(existing)
                        self.open.append(existing)

    def start(self):
        start_state, goal_state = self.read_input("astar_in.txt")

        # STEP 1: Initialize start node and put it in OPEN
        self.init_start_node(start_state, goal_state)

        # STEP 8: Loop until solution is found or failure
        while True:
            # STEP 2: Check if OPEN is empty
            if self.is_open_empty():
                # No solution (per strict request, we do not print debug); just exit
                return None, self.nodes_generated

            # STEP 3: Get node with smallest f from OPEN and move it to CLOSED
            n = self.get_best_node()

            # STEP 4: Check if n is the goal state
            if self.is_goal(n):
                path = self.reconstruct_path(n)
                self.display_solution(path)
                return path, self.nodes_generated

            # STEP 5: Expand node n, generate all children
            children = self.expand_node(n)

            # If there are no children, go back to Step 2
            if not children:
                continue

            # STEP 6 & 7: Insert children or update existing nodes
            self.add_children_to_open(children, n)


if __name__ == "__main__":
    heuristics = {
        "Manhattan Distance": manhattan_distance,
        "Misplaced Tiles": misplaced_tiles,
        "Nilsson's Sequence Score": nilsson_sequence_score
    }

    for name, func in heuristics.items():
        print("="*50)
        print(f"Using Heuristic: {name}")

        p = Puzzle(heuristic=func, heuristic_name=name)
        path, nodes = p.start()

        print(f"Nodes generated: {nodes}")
        print("="*50, "\n")

Using Heuristic: Manhattan Distance

Solution Found!
Number of moves: 18

Step 0: f(n)=12, g(n)=0, h(n)=12
2 1 6
4 * 8
7 5 3

Step 1: f(n)=12, g(n)=1, h(n)=11
2 1 6
4 8 *
7 5 3

Step 2: f(n)=12, g(n)=2, h(n)=10
2 1 *
4 8 6
7 5 3

Step 3: f(n)=14, g(n)=3, h(n)=11
2 * 1
4 8 6
7 5 3

Step 4: f(n)=16, g(n)=4, h(n)=12
2 8 1
4 * 6
7 5 3

Step 5: f(n)=16, g(n)=5, h(n)=11
2 8 1
4 6 *
7 5 3

Step 6: f(n)=16, g(n)=6, h(n)=10
2 8 1
4 6 3
7 5 *

Step 7: f(n)=16, g(n)=7, h(n)=9
2 8 1
4 6 3
7 * 5

Step 8: f(n)=16, g(n)=8, h(n)=8
2 8 1
4 * 3
7 6 5

Step 9: f(n)=16, g(n)=9, h(n)=7
2 8 1
* 4 3
7 6 5

Step 10: f(n)=18, g(n)=10, h(n)=8
* 8 1
2 4 3
7 6 5

Step 11: f(n)=18, g(n)=11, h(n)=7
8 * 1
2 4 3
7 6 5

Step 12: f(n)=18, g(n)=12, h(n)=6
8 1 *
2 4 3
7 6 5

Step 13: f(n)=18, g(n)=13, h(n)=5
8 1 3
2 4 *
7 6 5

Step 14: f(n)=18, g(n)=14, h(n)=4
8 1 3
2 * 4
7 6 5

Step 15: f(n)=18, g(n)=15, h(n)=3
8 1 3
* 2 4
7 6 5

Step 16: f(n)=18, g(n)=16, h(n)=2
* 1 3
8 2 4
7 6 5

Step 17: f(n)=18, g(n)=17, h(n)=1
1 * 